In [1]:
!pip install monai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 41.7 MB/s eta 0:00:00


In [2]:
!pip install tensorboardX

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.8 MB/s eta 0:00:00


In [3]:
import sys
import os
sys.path.append(os.path.abspath('/kaggle/input/gms-medical'))

In [4]:
# Basic Package
import torch
import argparse
import numpy as np
import yaml
import logging
import time
import os
from tqdm import tqdm
from torch.utils.data import DataLoader
from omegaconf import OmegaConf
from monai.losses.dice import DiceLoss

# Own Package
# from data.image_dataset import Image_Dataset
# from utils.tools import seed_reproducer, save_checkpoint, get_cuda, print_options
# from utils.get_logger import open_log
# from utils.lr_scheduler import LinearWarmupCosineAnnealingLR
from networks.latent_mapping_model import ResAttnUNet_DS, ResAttnUNet
from networks.models.autoencoder import AutoencoderKL
from networks.models.distributions import DiagonalGaussianDistribution

from tensorboardX import SummaryWriter


No module 'xformers'. Proceeding without it.


In [5]:
import os
import logging


def open_log(args, config):
    # open the log file
    log_savepath = '/kaggle/working/'
    if not os.path.exists(log_savepath):
        os.makedirs(log_savepath)
    log_name = args.config.split('/')[-1].split('.')[0]+'-yaml'
    if os.path.isfile(os.path.join(log_savepath, '{}.txt'.format(log_name))):
        os.remove(os.path.join(log_savepath, '{}.txt'.format(log_name)))
    initLogging(os.path.join(log_savepath, '{}.txt'.format(log_name)))


def initLogging(logFilename):
    """Init for logging
    """
    logging.basicConfig(
        level=logging.INFO,
        format='[%(asctime)s-%(levelname)s] %(message)s',
        datefmt='%y-%m-%d %H:%M:%S',
        filename=logFilename,
        filemode='w'
    )
    console = logging.StreamHandler()
    console.setLevel(logging.INFO)
    formatter = logging.Formatter('[%(asctime)s-%(levelname)s] %(message)s')
    console.setFormatter(formatter)
    logging.getLogger('').addHandler(console)


In [6]:
import math
import warnings
from typing import List

from torch.optim import Optimizer
from torch.optim.lr_scheduler import _LRScheduler

class LinearWarmupCosineAnnealingLR(_LRScheduler):

    def __init__(
        self,
        optimizer: Optimizer,
        warmup_epochs: int,
        max_epochs: int,
        warmup_start_lr: float = 0.0,
        eta_min: float = 0.0,
        last_epoch: int = -1,
    ) -> None:
        """
        Args:
            optimizer (Optimizer): Wrapped optimizer.
            warmup_epochs (int): Maximum number of iterations for linear warmup
            max_epochs (int): Maximum number of iterations
            warmup_start_lr (float): Learning rate to start the linear warmup. Default: 0.
            eta_min (float): Minimum learning rate. Default: 0.
            last_epoch (int): The index of last epoch. Default: -1.
        """
        self.warmup_epochs = warmup_epochs
        self.max_epochs = max_epochs
        self.warmup_start_lr = warmup_start_lr
        self.eta_min = eta_min

        super(LinearWarmupCosineAnnealingLR, self).__init__(optimizer, last_epoch)

    def get_lr(self) -> List[float]:
        """
        Compute learning rate using chainable form of the scheduler
        """
        if not self._get_lr_called_within_step:
            warnings.warn(
                "To get the last learning rate computed by the scheduler, "
                "please use `get_last_lr()`.",
                UserWarning,
            )

        if self.last_epoch == 0:
            return [self.warmup_start_lr] * len(self.base_lrs)
        elif self.last_epoch < self.warmup_epochs:
            return [
                group["lr"] + (base_lr - self.warmup_start_lr) / (self.warmup_epochs - 1)
                for base_lr, group in zip(self.base_lrs, self.optimizer.param_groups)
            ]
        elif self.last_epoch == self.warmup_epochs:
            return self.base_lrs
        elif (self.last_epoch - 1 - self.max_epochs) % (2 * (self.max_epochs - self.warmup_epochs)) == 0:
            return [
                group["lr"] + (base_lr - self.eta_min) *
                (1 - math.cos(math.pi / (self.max_epochs - self.warmup_epochs))) / 2
                for base_lr, group in zip(self.base_lrs, self.optimizer.param_groups)
            ]

        return [
            (1 + math.cos(math.pi * (self.last_epoch - self.warmup_epochs) / (self.max_epochs - self.warmup_epochs))) /
            (
                1 +
                math.cos(math.pi * (self.last_epoch - self.warmup_epochs - 1) / (self.max_epochs - self.warmup_epochs))
            ) * (group["lr"] - self.eta_min) + self.eta_min for group in self.optimizer.param_groups
        ]

    def _get_closed_form_lr(self) -> List[float]:
        """
        Called when epoch is passed as a param to the `step` function of the scheduler.
        """
        if self.last_epoch < self.warmup_epochs:
            return [
                self.warmup_start_lr + self.last_epoch * (base_lr - self.warmup_start_lr) / (self.warmup_epochs - 1)
                for base_lr in self.base_lrs
            ]

        return [
            self.eta_min + 0.5 * (base_lr - self.eta_min) *
            (1 + math.cos(math.pi * (self.last_epoch - self.warmup_epochs) / (self.max_epochs - self.warmup_epochs)))
            for base_lr in self.base_lrs
        ]

In [7]:
import torch
import os
import random
import numpy as np
import logging


def mkdir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def seed_reproducer(seed=2333):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.enabled = True


def load_checkpoint(model: torch.nn.Module, path: str) -> torch.nn.Module:
    if os.path.isfile(path):
        logging.info("=> loading checkpoint '{}'".format(path))
        
        # remap everthing onto CPU 
        state = torch.load(path, map_location=lambda storage, location: storage)

        # load weights
        model.load_state_dict(state['model'])
        logging.info("Loaded")
    else:
        model = None
        logging.info("=> no checkpoint found at '{}'".format(path))
    return model


def save_checkpoint(model: torch.nn.Module, save_name: str, path: str) -> None:
    model_savepath = os.path.join(path, 'checkpoints')
    if not os.path.exists(model_savepath):
        os.makedirs(model_savepath)
    file_name = os.path.join(model_savepath, save_name)
    torch.save({
        'model': model.state_dict(),
    },file_name)
    logging.info("save model to {}".format(file_name))


def adjust_learning_rate(optimizer, initial_lr, epoch, reduce_epoch, decay=0.5):
    lr = initial_lr * (decay ** (epoch // reduce_epoch))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    logging.info('Change Learning Rate to {}'.format(lr))
    return lr


def get_cuda(tensor):
    if torch.cuda.is_available():
        tensor = tensor.cuda()
    return tensor


def print_options(configs):
    message = ''
    message += '----------------- Options ---------------\n'
    for k, v in configs.items():
        message += '{:>25}: {:<30}\n'.format(str(k), str(v))
    message += '----------------- End -------------------'
    logging.info(message)

    # save to the disk
    file_name = os.path.join(configs['log_path'], '{}_configs.txt'.format(configs['phase']))
    with open(file_name, 'wt') as opt_file:
        opt_file.write(message)
        opt_file.write('\n')

In [8]:
import os
import pickle
import logging
import numpy as np
import albumentations as A
from PIL import Image
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset
import re

class Image_Dataset(Dataset):
    def __init__(self, pickle_file_path, stage='train') -> None:
        super().__init__()
        with open(pickle_file_path, 'rb') as file:
            loaded_dict = pickle.load(file)
        self.img_path          = os.path.join(os.path.dirname(pickle_file_path), 'images')
        self.mask_path         = os.path.join(os.path.dirname(pickle_file_path), 'masks')
        self.img_size          = 224
        self.stage             = stage
        self.name_list         = loaded_dict[stage]['name_list']
        self.transform         = self.get_transforms()
        logging.info('{} set num: {}'.format(stage, len(self.name_list)))

        del loaded_dict

    def get_transforms(self):
        if self.stage == 'train':
            transforms = A.Compose([
                A.ToFloat(max_value=255.0, always_apply=True),
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.RandomRotate90(p=0.5),
                A.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1, p=0.2),
                A.ShiftScaleRotate(shift_limit=0.15, scale_limit=0.1, rotate_limit=20, p=0.4),
                A.Resize(self.img_size, self.img_size, always_apply=True),
                ToTensorV2(always_apply=True),
            ])
        else:
            transforms = A.Compose([
                A.ToFloat(max_value=255.0, always_apply=True),
                A.Resize(self.img_size, self.img_size, always_apply=True),
                ToTensorV2(always_apply=True),
            ])
        return transforms

    def __getitem__(self, index):
        name = self.name_list[index]
        name = re.sub(r"\D", "", name)
        seg_image = Image.open(os.path.join(self.mask_path, name + '.png')).convert("RGB")
        seg_data  = np.array(seg_image).astype(np.float32)
        img_image = Image.open(os.path.join(self.img_path,  name + '.png')).convert("RGB")
        img_data  = np.array(img_image).astype(np.float32)

        augmented = self.transform(image=img_data, mask=seg_data)

        aug_img = augmented['image']
        aug_seg = augmented['mask']

        return {
            'name': name,
            'img': aug_img,
            'seg': aug_seg
        }

    def __len__(self):
        return len(self.name_list)


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [9]:

# Set GPU ID
gpus = ','.join([str(i) for i in [0]])
os.environ["CUDA_VISIBLE_DEVICES"] = gpus
    
writer = SummaryWriter('./logs/')

train_dataset = Image_Dataset('/kaggle/input/gms-medical/Dataset/glas/glas_train_test_names.pkl', stage='train')
valid_dataset = Image_Dataset('/kaggle/input/gms-medical/Dataset/glas/glas_train_test_names.pkl', stage='test')
train_dataloader = DataLoader(train_dataset, batch_size=2, pin_memory=True, drop_last=True, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=2, pin_memory=True, drop_last=False, shuffle=False)

mapping_model = get_cuda(ResAttnUNet_DS(
    in_channel=4, 
    out_channels=4, 
    num_res_blocks=2, 
    ch=32, 
    ch_mult=[1,2,4,4]
))

# get VAE (first-stage model)
vae_path = '/kaggle/input/gms-medical/configs/v2-inference-v-first-stage-VAE.yaml'
vae_config = OmegaConf.load(f"{vae_path}")
vae_model = AutoencoderKL(**vae_config.first_stage_config.get("params", dict()))


making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


In [10]:
def get_multi_loss(criterion, out_dict, label, is_ds=True, key_list=None):
        keys = key_list if key_list is not None else list(out_dict.keys())
        if is_ds:
            multi_loss = sum([criterion(out_dict[key], label) for key in keys])
        else:
            multi_loss = criterion(out_dict['out'], label)
        return multi_loss

In [11]:
def get_vae_encoding_mu_and_sigma(encoder_posterior, scale_factor):
    if isinstance(encoder_posterior, DiagonalGaussianDistribution):
        mean, logvar = encoder_posterior.mu_and_sigma()
    else:
        raise NotImplementedError(f"encoder_posterior of type '{type(encoder_posterior)}' not yet implemented")
    return scale_factor * mean, logvar

In [12]:
def vae_decode(vae_model, pred_mean, scale_factor):
    z = 1. / scale_factor * pred_mean
    pred_seg = vae_model.decode(z)
    pred_seg = torch.mean(pred_seg, dim=1, keepdim=True)
    pred_seg = torch.clamp((pred_seg + 1.0) / 2.0, min=0.0, max=1.0) # (B, 1, H, W)
    return pred_seg

In [13]:
def arg_parse() -> argparse.ArgumentParser.parse_args :
    parser = argparse.ArgumentParser()
    parser.add_argument('--config', default='./configs/train.yaml',
                        type=str, help='load the config file')
    args = parser.parse_args()
    return args

In [14]:
pl_sd = torch.load("/kaggle/input/gms-medical/SD-VAE-weights/768-v-ema-first-stage-VAE.ckpt", map_location="cpu")
sd = pl_sd["state_dict"]
vae_model.load_state_dict(sd, strict=True)

vae_model.freeze()
vae_model = get_cuda(vae_model)
scale_factor = vae_config.first_stage_config.scale_factor


<ipython-input-14-7bb22cda0f6f>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pl_sd = torch.load("/kaggle/input/gms-medical/SD-VAE-weights/768-v-ema-first-stage-VAE.ckpt"

In [15]:
import torch
from torch import nn
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
import torch.nn.functional as F
import os
import matplotlib.pyplot as plt
# from utils import *
from typing import List, Tuple, Union
import timm
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
import types
import math
from abc import ABCMeta, abstractmethod
# from mmcv.cnn import ConvModule
from pdb import set_trace as st

from torch.nn import init


class KANLinear(torch.nn.Module):
    def __init__(
        self,
        in_features,
        out_features,
        grid_size=5,
        spline_order=3,
        scale_noise=0.1,
        scale_base=1.0,
        scale_spline=1.0,
        enable_standalone_scale_spline=True,
        base_activation=torch.nn.SiLU,
        grid_eps=0.02,
        grid_range=[-1, 1],
    ):
        super(KANLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.grid_size = grid_size
        self.spline_order = spline_order

        h = (grid_range[1] - grid_range[0]) / grid_size
        grid = (
            (
                torch.arange(-spline_order, grid_size + spline_order + 1) * h
                + grid_range[0]
            )
            .expand(in_features, -1)
            .contiguous()
        )
        self.register_buffer("grid", grid)

        self.base_weight = torch.nn.Parameter(torch.Tensor(out_features, in_features))
        self.spline_weight = torch.nn.Parameter(
            torch.Tensor(out_features, in_features, grid_size + spline_order)
        )
        if enable_standalone_scale_spline:
            self.spline_scaler = torch.nn.Parameter(
                torch.Tensor(out_features, in_features)
            )

        self.scale_noise = scale_noise
        self.scale_base = scale_base
        self.scale_spline = scale_spline
        self.enable_standalone_scale_spline = enable_standalone_scale_spline
        self.base_activation = base_activation()
        self.grid_eps = grid_eps

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.kaiming_uniform_(self.base_weight, a=math.sqrt(5) * self.scale_base)
        with torch.no_grad():
            noise = (
                (
                    torch.rand(self.grid_size + 1, self.in_features, self.out_features)
                    - 1 / 2
                )
                * self.scale_noise
                / self.grid_size
            )
            self.spline_weight.data.copy_(
                (self.scale_spline if not self.enable_standalone_scale_spline else 1.0)
                * self.curve2coeff(
                    self.grid.T[self.spline_order : -self.spline_order],
                    noise,
                )
            )
            if self.enable_standalone_scale_spline:
                # torch.nn.init.constant_(self.spline_scaler, self.scale_spline)
                torch.nn.init.kaiming_uniform_(self.spline_scaler, a=math.sqrt(5) * self.scale_spline)

    def b_splines(self, x: torch.Tensor):
        """
        Compute the B-spline bases for the given input tensor.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_features).

        Returns:
            torch.Tensor: B-spline bases tensor of shape (batch_size, in_features, grid_size + spline_order).
        """
        assert x.dim() == 2 and x.size(1) == self.in_features

        grid: torch.Tensor = (
            self.grid
        )  # (in_features, grid_size + 2 * spline_order + 1)
        x = x.unsqueeze(-1)
        bases = ((x >= grid[:, :-1]) & (x < grid[:, 1:])).to(x.dtype)
        for k in range(1, self.spline_order + 1):
            bases = (
                (x - grid[:, : -(k + 1)])
                / (grid[:, k:-1] - grid[:, : -(k + 1)])
                * bases[:, :, :-1]
            ) + (
                (grid[:, k + 1 :] - x)
                / (grid[:, k + 1 :] - grid[:, 1:(-k)])
                * bases[:, :, 1:]
            )

        assert bases.size() == (
            x.size(0),
            self.in_features,
            self.grid_size + self.spline_order,
        )
        return bases.contiguous()

    def curve2coeff(self, x: torch.Tensor, y: torch.Tensor):
        """
        Compute the coefficients of the curve that interpolates the given points.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_features).
            y (torch.Tensor): Output tensor of shape (batch_size, in_features, out_features).

        Returns:
            torch.Tensor: Coefficients tensor of shape (out_features, in_features, grid_size + spline_order).
        """
        assert x.dim() == 2 and x.size(1) == self.in_features
        assert y.size() == (x.size(0), self.in_features, self.out_features)

        A = self.b_splines(x).transpose(
            0, 1
        )  # (in_features, batch_size, grid_size + spline_order)
        B = y.transpose(0, 1)  # (in_features, batch_size, out_features)
        solution = torch.linalg.lstsq(
            A, B
        ).solution  # (in_features, grid_size + spline_order, out_features)
        result = solution.permute(
            2, 0, 1
        )  # (out_features, in_features, grid_size + spline_order)

        assert result.size() == (
            self.out_features,
            self.in_features,
            self.grid_size + self.spline_order,
        )
        return result.contiguous()

    @property
    def scaled_spline_weight(self):
        return self.spline_weight * (
            self.spline_scaler.unsqueeze(-1)
            if self.enable_standalone_scale_spline
            else 1.0
        )

    def forward(self, x: torch.Tensor):
        assert x.dim() == 2 and x.size(1) == self.in_features

        base_output = F.linear(self.base_activation(x), self.base_weight)
        spline_output = F.linear(
            self.b_splines(x).view(x.size(0), -1),
            self.scaled_spline_weight.view(self.out_features, -1),
        )
        return base_output + spline_output

    @torch.no_grad()
    def update_grid(self, x: torch.Tensor, margin=0.01):
        assert x.dim() == 2 and x.size(1) == self.in_features
        batch = x.size(0)

        splines = self.b_splines(x)  # (batch, in, coeff)
        splines = splines.permute(1, 0, 2)  # (in, batch, coeff)
        orig_coeff = self.scaled_spline_weight  # (out, in, coeff)
        orig_coeff = orig_coeff.permute(1, 2, 0)  # (in, coeff, out)
        unreduced_spline_output = torch.bmm(splines, orig_coeff)  # (in, batch, out)
        unreduced_spline_output = unreduced_spline_output.permute(
            1, 0, 2
        )  # (batch, in, out)

        # sort each channel individually to collect data distribution
        x_sorted = torch.sort(x, dim=0)[0]
        grid_adaptive = x_sorted[
            torch.linspace(
                0, batch - 1, self.grid_size + 1, dtype=torch.int64, device=x.device
            )
        ]

        uniform_step = (x_sorted[-1] - x_sorted[0] + 2 * margin) / self.grid_size
        grid_uniform = (
            torch.arange(
                self.grid_size + 1, dtype=torch.float32, device=x.device
            ).unsqueeze(1)
            * uniform_step
            + x_sorted[0]
            - margin
        )

        grid = self.grid_eps * grid_uniform + (1 - self.grid_eps) * grid_adaptive
        grid = torch.concatenate(
            [
                grid[:1]
                - uniform_step
                * torch.arange(self.spline_order, 0, -1, device=x.device).unsqueeze(1),
                grid,
                grid[-1:]
                + uniform_step
                * torch.arange(1, self.spline_order + 1, device=x.device).unsqueeze(1),
            ],
            dim=0,
        )

        self.grid.copy_(grid.T)
        self.spline_weight.data.copy_(self.curve2coeff(x, unreduced_spline_output))

    def regularization_loss(self, regularize_activation=1.0, regularize_entropy=1.0):
        """
        Compute the regularization loss.

        This is a dumb simulation of the original L1 regularization as stated in the
        paper, since the original one requires computing absolutes and entropy from the
        expanded (batch, in_features, out_features) intermediate tensor, which is hidden
        behind the F.linear function if we want an memory efficient implementation.

        The L1 regularization is now computed as mean absolute value of the spline
        weights. The authors implementation also includes this term in addition to the
        sample-based regularization.
        """
        l1_fake = self.spline_weight.abs().mean(-1)
        regularization_loss_activation = l1_fake.sum()
        p = l1_fake / regularization_loss_activation
        regularization_loss_entropy = -torch.sum(p * p.log())
        return (
            regularize_activation * regularization_loss_activation
            + regularize_entropy * regularization_loss_entropy
        )

class KANLayer(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0., no_kan=False):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.dim = in_features
        
        grid_size=5
        spline_order=3
        scale_noise=0.1
        scale_base=1.0
        scale_spline=1.0
        base_activation=torch.nn.SiLU
        grid_eps=0.02
        grid_range=[-1, 1]

        if not no_kan:
            self.fc1 = KANLinear(
                        in_features,
                        hidden_features,
                        grid_size=grid_size,
                        spline_order=spline_order,
                        scale_noise=scale_noise,
                        scale_base=scale_base,
                        scale_spline=scale_spline,
                        base_activation=base_activation,
                        grid_eps=grid_eps,
                        grid_range=grid_range,
                    )
            self.fc2 = KANLinear(
                        hidden_features,
                        out_features,
                        grid_size=grid_size,
                        spline_order=spline_order,
                        scale_noise=scale_noise,
                        scale_base=scale_base,
                        scale_spline=scale_spline,
                        base_activation=base_activation,
                        grid_eps=grid_eps,
                        grid_range=grid_range,
                    )
            self.fc3 = KANLinear(
                        hidden_features,
                        out_features,
                        grid_size=grid_size,
                        spline_order=spline_order,
                        scale_noise=scale_noise,
                        scale_base=scale_base,
                        scale_spline=scale_spline,
                        base_activation=base_activation,
                        grid_eps=grid_eps,
                        grid_range=grid_range,
                    )
            # # TODO   
            # self.fc4 = KANLinear(
            #             hidden_features,
            #             out_features,
            #             grid_size=grid_size,
            #             spline_order=spline_order,
            #             scale_noise=scale_noise,
            #             scale_base=scale_base,
            #             scale_spline=scale_spline,
            #             base_activation=base_activation,
            #             grid_eps=grid_eps,
            #             grid_range=grid_range,
            #         )   

        else:
            self.fc1 = nn.Linear(in_features, hidden_features)
            self.fc2 = nn.Linear(hidden_features, out_features)
            self.fc3 = nn.Linear(hidden_features, out_features)

        # TODO
        # self.fc1 = nn.Linear(in_features, hidden_features)


        self.dwconv_1 = DW_bn_relu(hidden_features)
        self.dwconv_2 = DW_bn_relu(hidden_features)
        self.dwconv_3 = DW_bn_relu(hidden_features)

        # # TODO
        # self.dwconv_4 = DW_bn_relu(hidden_features)
    
        self.drop = nn.Dropout(drop)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()
    

    def forward(self, x, H, W):
        # pdb.set_trace()
        B, N, C = x.shape

        x = self.fc1(x.reshape(B*N,C))
        x = x.reshape(B,N,C).contiguous()
        x = self.dwconv_1(x, H, W)
        x = self.fc2(x.reshape(B*N,C))
        x = x.reshape(B,N,C).contiguous()
        x = self.dwconv_2(x, H, W)
        x = self.fc3(x.reshape(B*N,C))
        x = x.reshape(B,N,C).contiguous()
        x = self.dwconv_3(x, H, W)

        # # TODO
        # x = x.reshape(B,N,C).contiguous()
        # x = self.dwconv_4(x, H, W)
    
        return x

class KANBlock(nn.Module):
    def __init__(self, dim, drop=0., drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm, no_kan=False):
        super().__init__()

        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim)

        self.layer = KANLayer(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop, no_kan=no_kan)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    def forward(self, x, H, W):
        x = x + self.drop_path(self.layer(self.norm2(x), H, W))

        return x


class PolynomialFunction(nn.Module):
    def __init__(self, 
                 degree: int = 3):
        super().__init__()
        self.degree = degree

    def forward(self, x):
        return torch.stack([x ** i for i in range(self.degree)], dim=-1)
    
class BSplineFunction(nn.Module):
    def __init__(self, grid_min: float = -2.,
        grid_max: float = 2., degree: int = 3, num_basis: int = 8):
        super(BSplineFunction, self).__init__()
        self.degree = degree
        self.num_basis = num_basis
        self.knots = torch.linspace(grid_min, grid_max, num_basis + degree + 1)  # Uniform knots

    def basis_function(self, i, k, t):
        if k == 0:
            return ((self.knots[i] <= t) & (t < self.knots[i + 1])).float()
        else:
            left_num = (t - self.knots[i]) * self.basis_function(i, k - 1, t)
            left_den = self.knots[i + k] - self.knots[i]
            left = left_num / left_den if left_den != 0 else 0

            right_num = (self.knots[i + k + 1] - t) * self.basis_function(i + 1, k - 1, t)
            right_den = self.knots[i + k + 1] - self.knots[i + 1]
            right = right_num / right_den if right_den != 0 else 0
            return left + right 
    
    def forward(self, x):
        x = x.squeeze()  # Assuming x is of shape (B, 1)
        basis_functions = torch.stack([self.basis_function(i, self.degree, x) for i in range(self.num_basis)], dim=-1)
        return basis_functions

class ChebyshevFunction(nn.Module):
    def __init__(self, degree: int = 4):
        super(ChebyshevFunction, self).__init__()
        self.degree = degree

    def forward(self, x):
        chebyshev_polynomials = [torch.ones_like(x), x]
        for n in range(2, self.degree):
            chebyshev_polynomials.append(2 * x * chebyshev_polynomials[-1] - chebyshev_polynomials[-2])
        return torch.stack(chebyshev_polynomials, dim=-1)

class FourierBasisFunction(nn.Module):
    def __init__(self, 
                 num_frequencies: int = 4, 
                 period: float = 1.0):
        super(FourierBasisFunction, self).__init__()
        assert num_frequencies % 2 == 0, "num_frequencies must be even"
        self.num_frequencies = num_frequencies
        self.period = nn.Parameter(torch.Tensor([period]), requires_grad=False)

    def forward(self, x):
        frequencies = torch.arange(1, self.num_frequencies // 2 + 1, device=x.device)
        sin_components = torch.sin(2 * torch.pi * frequencies * x[..., None] / self.period)
        cos_components = torch.cos(2 * torch.pi * frequencies * x[..., None] / self.period)
        basis_functions = torch.cat([sin_components, cos_components], dim=-1)
        return basis_functions
        
class RadialBasisFunction(nn.Module):
    def __init__(
        self,
        grid_min: float = -2.,
        grid_max: float = 2.,
        num_grids: int = 4,
        denominator: float = None,
    ):
        super().__init__()
        grid = torch.linspace(grid_min, grid_max, num_grids)
        self.grid = torch.nn.Parameter(grid, requires_grad=False)
        self.denominator = denominator or (grid_max - grid_min) / (num_grids - 1)

    def forward(self, x):
        return torch.exp(-((x[..., None] - self.grid) / self.denominator) ** 2)
    

    
    
class SplineConv2D(nn.Conv2d):
    def __init__(self, 
                 in_channels: int, 
                 out_channels: int, 
                 kernel_size: Union[int, Tuple[int, int]] = 3,
                 stride: Union[int, Tuple[int, int]] = 1, 
                 padding: Union[int, Tuple[int, int]] = 0, 
                 dilation: Union[int, Tuple[int, int]] = 1,
                 groups: int = 1, 
                 bias: bool = True, 
                 init_scale: float = 0.1, 
                 padding_mode: str = "zeros", 
                 **kw
                 ) -> None:
        self.init_scale = init_scale
        super().__init__(in_channels, 
                         out_channels, 
                         kernel_size, 
                         stride, 
                         padding, 
                         dilation, 
                         groups, 
                         bias, 
                         padding_mode, 
                         **kw
                         )

    def reset_parameters(self) -> None:
        nn.init.trunc_normal_(self.weight, mean=0, std=self.init_scale)
        if self.bias is not None:
            nn.init.zeros_(self.bias)


class FastKANConvLayer(nn.Module):
    def __init__(self, 
                 in_channels: int, 
                 out_channels: int, 
                 kernel_size: Union[int, Tuple[int, int]] = 3,
                 stride: Union[int, Tuple[int, int]] = 1, 
                 padding: Union[int, Tuple[int, int]] = 0, 
                 dilation: Union[int, Tuple[int, int]] = 1,
                 groups: int = 1, 
                 bias: bool = True, 
                 grid_min: float = -2., 
                 grid_max: float = 2.,
                 num_grids: int = 4, 
                 use_base_update: bool = True, 
                 base_activation = F.silu,
                 spline_weight_init_scale: float = 0.1, 
                 padding_mode: str = "zeros",
                 kan_type: str = "BSpline",
                #  kan_type: str = "RBF",
                 ) -> None:
        
        super().__init__()
        if kan_type == "RBF":
            self.rbf = RadialBasisFunction(grid_min, grid_max, num_grids)
        elif kan_type == "Fourier":
            self.rbf = FourierBasisFunction(num_grids)
        elif kan_type == "Poly":
            self.rbf = PolynomialFunction(num_grids)
        elif kan_type == "Chebyshev":
            self.rbf = ChebyshevFunction(num_grids)
        elif kan_type == "BSpline":
            self.rbf = BSplineFunction(grid_min, grid_max, 3, num_grids)

        self.spline_conv = SplineConv2D(in_channels * num_grids, 
                                        out_channels, 
                                        kernel_size,
                                        stride, 
                                        padding, 
                                        dilation, 
                                        groups, 
                                        bias,
                                        spline_weight_init_scale, 
                                        padding_mode)
        
        self.use_base_update = use_base_update
        if use_base_update:
            self.base_activation = base_activation
            self.base_conv = nn.Conv2d(in_channels, 
                                       out_channels, 
                                       kernel_size, 
                                       stride, 
                                       padding, 
                                       dilation, 
                                       groups, 
                                       bias, 
                                       padding_mode)

    def forward(self, x):
        batch_size, channels, height, width = x.shape
        x_rbf = self.rbf(x.view(batch_size, channels, -1)).view(batch_size, channels, height, width, -1)
        x_rbf = x_rbf.permute(0, 4, 1, 2, 3).contiguous().view(batch_size, -1, height, width)
        
        # Apply spline convolution
        ret = self.spline_conv(x_rbf)
         
        if self.use_base_update:
            base = self.base_conv(self.base_activation(x))
            ret = ret + base
        
        return ret


class DWConv(nn.Module):
    def __init__(self, dim=768):
        super(DWConv, self).__init__()
        self.dwconv = nn.Conv2d(dim, dim, 3, 1, 1, bias=True, groups=dim)

    def forward(self, x, H, W):
        B, N, C = x.shape
        x = x.transpose(1, 2).view(B, C, H, W)
        x = self.dwconv(x)
        x = x.flatten(2).transpose(1, 2)

        return x

class DW_bn_relu(nn.Module):
    def __init__(self, dim=768):
        super(DW_bn_relu, self).__init__()
        self.dwconv = nn.Conv2d(dim, dim, 3, 1, 1, bias=True, groups=dim)
        self.bn = nn.BatchNorm2d(dim)
        self.relu = nn.ReLU()

    def forward(self, x, H, W):
        B, N, C = x.shape
        x = x.transpose(1, 2).view(B, C, H, W)
        x = self.dwconv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = x.flatten(2).transpose(1, 2)

        return x

class PatchEmbed(nn.Module):
    """ Image to Patch Embedding
    """

    def __init__(self, img_size=224, patch_size=7, stride=4, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)

        self.img_size = img_size
        self.patch_size = patch_size
        self.H, self.W = img_size[0] // patch_size[0], img_size[1] // patch_size[1]
        self.num_patches = self.H * self.W
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=stride,
                              padding=(patch_size[0] // 2, patch_size[1] // 2))
        self.norm = nn.LayerNorm(embed_dim)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    def forward(self, x):
        x = self.proj(x)
        _, _, H, W = x.shape
        x = x.flatten(2).transpose(1, 2)
        x = self.norm(x)

        return x, H, W


class KConvLayer(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(KConvLayer, self).__init__()
        self.conv = FastKANConvLayer(in_ch, out_ch, 3, padding=1, kan_type = "BSpline", num_grids=4, grid_min=-2., grid_max=2.)
        self.bn1 = nn.BatchNorm2d(out_ch)
        
    def forward(self, input):
        output1 = self.conv(input)
        output1 = self.bn1(output1)
        return output1
    
class ConvLayer(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(ConvLayer, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, input):
        return self.conv(input)

class KD_ConvLayer(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(KD_ConvLayer, self).__init__()
        self.conv = FastKANConvLayer(in_ch, out_ch, 3, padding=1, kan_type = "BSpline", num_grids=4, grid_min=-2., grid_max=2.)
        self.bn1 = nn.BatchNorm2d(out_ch)
        
    def forward(self, input):
        output1 = self.conv(input)
        output1 = self.bn1(output1)
        return output1
    
class D_ConvLayer(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(D_ConvLayer, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, in_ch, 3, padding=1),
            nn.BatchNorm2d(in_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, input):
        return self.conv(input)

/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [16]:
def Normalize(in_channels):
    return torch.nn.GroupNorm(num_groups=8, num_channels=in_channels, eps=1e-6, affine=True)

In [17]:
class UKAN(nn.Module):
    def __init__(self, input_channels=8, deep_supervision=False, img_size=32, out_channels = 4 , patch_size=16, in_chans=8, embed_dims=[256, 320, 512], no_kan=False,
                 drop_rate=0., drop_path_rate=0., norm_layer=nn.LayerNorm, depths=[1, 1, 1], **kwargs):
        super().__init__()

        kan_input_dim = embed_dims[0]
        self.input_blocks = nn.Conv2d(input_channels, in_chans, kernel_size=3, stride=1, padding=1, bias=True)

        # Update input channels to 8
        self.encoder1 = ConvLayer(input_channels, kan_input_dim // 4)
        self.encoder2 = ConvLayer(kan_input_dim // 4, kan_input_dim // 2)
        self.encoder3 = ConvLayer(kan_input_dim // 2, kan_input_dim)

        self.norm3 = norm_layer(embed_dims[1])
        self.norm4 = norm_layer(embed_dims[2])

        self.dnorm3 = norm_layer(embed_dims[1])
        self.dnorm4 = norm_layer(embed_dims[0])

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]

        self.block1 = nn.ModuleList([KANBlock(
            dim=embed_dims[1],
            drop=drop_rate, drop_path=dpr[0], norm_layer=norm_layer
        )])

        self.block2 = nn.ModuleList([KANBlock(
            dim=embed_dims[2],
            drop=drop_rate, drop_path=dpr[1], norm_layer=norm_layer
        )])

        self.dblock1 = nn.ModuleList([KANBlock(
            dim=embed_dims[1],
            drop=drop_rate, drop_path=dpr[0], norm_layer=norm_layer
        )])

        self.dblock2 = nn.ModuleList([KANBlock(
            dim=embed_dims[0],
            drop=drop_rate, drop_path=dpr[1], norm_layer=norm_layer
        )])

        # Adjust patch embedding for smaller input size
        self.patch_embed3 = PatchEmbed(img_size=img_size, patch_size=3, stride=1, in_chans=embed_dims[0], embed_dim=embed_dims[1])
        self.patch_embed4 = PatchEmbed(img_size=img_size, patch_size=3, stride=1, in_chans=embed_dims[1], embed_dim=embed_dims[2])

        self.decoder1 = D_ConvLayer(embed_dims[2], embed_dims[1])
        self.decoder2 = D_ConvLayer(embed_dims[1], embed_dims[0])
        self.decoder3 = D_ConvLayer(embed_dims[0], embed_dims[0] // 2)
        self.decoder4 = D_ConvLayer(embed_dims[0] // 2, embed_dims[0] // 4)
        self.decoder5 = D_ConvLayer(embed_dims[0] // 4, embed_dims[0] // 4)

        self.final = nn.Conv2d(embed_dims[0] // 8, 4, kernel_size=3, stride=1, padding=1)
        

        self.convds3 = nn.Sequential(Normalize(128), nn.SiLU(),
                                     nn.Conv2d(128, out_channels, kernel_size=3, stride=1, padding=1, bias=False))
        self.convds2 = nn.Sequential(Normalize(64), nn.SiLU(),
                                     nn.Conv2d(64, out_channels, kernel_size=3, stride=1, padding=1, bias=False))
        self.convds1 = nn.Sequential(Normalize(16), nn.SiLU(),
                                     nn.Conv2d(16, out_channels, kernel_size=3, stride=1, padding=1, bias=False))
        self.convds0 = nn.Sequential(Normalize(16), nn.SiLU(),
                                     nn.Conv2d(16, out_channels, kernel_size=3, stride=1, padding=1, bias=False))      
        
        # self.soft = nn.Softmax(dim=1)

    def forward(self, x):
        B = x.shape[0]

        ### Encoder
        ### Stage 1
        out = F.relu(self.encoder1(x))
        t1 = out
        ### Stage 2
        out = F.relu(self.encoder2(out))
        t2 = out
        ### Stage 3
        out = F.relu(self.encoder3(out))
        t3 = out

        ### Tokenized KAN Stage
        ### Stage 4
        out, H, W = self.patch_embed3(out)
        for i, blk in enumerate(self.block1):
            out = blk(out, H, W)
        out = self.norm3(out)
        out = out.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()
        t4 = out

        ### Bottleneck
        out, H, W = self.patch_embed4(out)
        for i, blk in enumerate(self.block2):
            out = blk(out, H, W)
        out = self.norm4(out)
        out = out.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()

        ### Decoder
        ### Stage 4
        out_final = dict()
        out = F.relu(F.interpolate(self.decoder1(out), scale_factor=(1, 1), mode='bilinear'))
        out_final['level3'] = self.convds3(out)
        out = torch.add(out, t4)
        _, _, H, W = out.shape
        out = out.flatten(2).transpose(1, 2)
        for i, blk in enumerate(self.dblock1):
            out = blk(out, H, W)

        ### Stage 3
        out = self.dnorm3(out)
        out = out.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()
        out = F.relu(F.interpolate(self.decoder2(out), scale_factor=(1, 1), mode='bilinear'))
        out_final['level2'] = self.convds2(out)
        out = torch.add(out, t3)
        _, _, H, W = out.shape
        out = out.flatten(2).transpose(1, 2)

        for i, blk in enumerate(self.dblock2):
            out = blk(out, H, W)

        out = self.dnorm4(out)
        out = out.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()

        out = F.relu(F.interpolate(self.decoder3(out), scale_factor=(1, 1), mode='bilinear'))
        out = torch.add(out, t2)
        out = F.relu(F.interpolate(self.decoder4(out), scale_factor=(1, 1), mode='bilinear'))
        out_final['level1'] = self.convds1(out)
        out = torch.add(out, t1)
        out = F.relu(F.interpolate(self.decoder5(out), scale_factor=(1, 1), mode='bilinear'))
        out_final['out'] = self.convds0(out)

        return out_final

In [18]:
model = UKAN(input_channels=4, deep_supervision=False, embed_dims=[64, 128, 256], no_kan=False)

In [19]:
import torch
import logging
from torch import nn
from einops import rearrange

In [20]:
class KSpatialSelfAttention(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.in_channels = in_channels

        self.norm = Normalize(in_channels)
        self.q = FastKANConvLayer(in_channels, in_channels, 1, stride=1, padding=0, kan_type = "BSpline", num_grids=4, grid_min=-2., grid_max=2.)
        self.k = FastKANConvLayer(in_channels, in_channels, 1, stride=1, padding=0, kan_type = "BSpline", num_grids=4, grid_min=-2., grid_max=2.)
        self.v = FastKANConvLayer(in_channels, in_channels, 1, stride=1, padding=0, kan_type = "BSpline", num_grids=4, grid_min=-2., grid_max=2.)
        self.proj_out = FastKANConvLayer(in_channels, in_channels, 1, stride=1, padding=0, kan_type = "BSpline", num_grids=4, grid_min=-2., grid_max=2.)

    def forward(self, x):
        h_ = x
        h_ = self.norm(h_)
        q = self.q(h_)
        k = self.k(h_)
        v = self.v(h_)

        # compute attention
        b,c,h,w = q.shape
        q = rearrange(q, 'b c h w -> b (h w) c')
        k = rearrange(k, 'b c h w -> b c (h w)')
        w_ = torch.einsum('bij,bjk->bik', q, k)

        w_ = w_ * (int(c)**(-0.5))
        w_ = torch.nn.functional.softmax(w_, dim=2)

        # attend to values
        v = rearrange(v, 'b c h w -> b c (h w)')
        w_ = rearrange(w_, 'b i j -> b j i')
        h_ = torch.einsum('bij,bjk->bik', v, w_)
        h_ = rearrange(h_, 'b c (h w) -> b c h w', h=h)
        h_ = self.proj_out(h_)

        return x+h_

In [21]:
class KResBlock(nn.Module):
    """
    Convolutional blocks
    """
    def __init__(self, in_channels, out_channels, leaky=True):
        super().__init__()
        # activation, support PReLU and common ReLU
        self.act1 = nn.PReLU() if leaky else nn.ReLU(inplace=True)
        self.act2 = nn.PReLU() if leaky else nn.ReLU(inplace=True)

        self.conv1 = nn.Sequential(
            Normalize(in_channels),
            # self.act1,
            FastKANConvLayer(in_channels, out_channels, 3, stride=1, padding=1, kan_type = "BSpline", num_grids=4, grid_min=-2., grid_max=2.)
            # nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True),
        )
        
        self.conv2 = nn.Sequential(
            Normalize(out_channels),
            # self.act2,
            FastKANConvLayer(out_channels, out_channels, 3, stride=1, padding=1, kan_type = "BSpline", num_grids=4, grid_min=-2., grid_max=2.)
            # nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True),
        )

        if in_channels == out_channels:
            self.skip_connection = nn.Identity()
        else:
            self.skip_connection = FastKANConvLayer(in_channels, out_channels, 1, stride=1, padding=0, kan_type = "BSpline", num_grids=4, grid_min=-2., grid_max=2.)
            # self.skip_connection = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)


    def forward(self, x):
        # print('x2', x.shape)
        h = self.conv1(x)
        # print('h2', h.shape)
        h = self.conv2(h)
        # print('h3', h.shape)
        return self.skip_connection(x) + h
    
class KResAttBlock(nn.Module):
    """
    Convolutional blocks
    """
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.resblock  = KResBlock(in_channels=in_channels, out_channels=out_channels)
        self.attention = SpatialSelfAttention(out_channels)


    def forward(self, x):
        # print('x', x.shape)
        h = self.resblock(x)
        # print('h1', h.shape)
        h = self.attention(h)
        return h

In [22]:
import torch
import logging
from torch import nn
from einops import rearrange


def Normalize(in_channels):
    return torch.nn.GroupNorm(num_groups=16, num_channels=in_channels, eps=1e-6, affine=True)
    # return torch.nn.Identity()


class SpatialSelfAttention(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.in_channels = in_channels

        self.norm = Normalize(in_channels)
        self.q = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.k = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.v = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.proj_out = torch.nn.Conv2d(in_channels,
                                        in_channels,
                                        kernel_size=1,
                                        stride=1,
                                        padding=0)

    def forward(self, x):
        h_ = x
        h_ = self.norm(h_)
        q = self.q(h_)
        k = self.k(h_)
        v = self.v(h_)

        # compute attention
        b,c,h,w = q.shape
        q = rearrange(q, 'b c h w -> b (h w) c')
        k = rearrange(k, 'b c h w -> b c (h w)')
        w_ = torch.einsum('bij,bjk->bik', q, k)

        w_ = w_ * (int(c)**(-0.5))
        w_ = torch.nn.functional.softmax(w_, dim=2)

        # attend to values
        v = rearrange(v, 'b c h w -> b c (h w)')
        w_ = rearrange(w_, 'b i j -> b j i')
        h_ = torch.einsum('bij,bjk->bik', v, w_)
        h_ = rearrange(h_, 'b c (h w) -> b c h w', h=h)
        h_ = self.proj_out(h_)

        return x+h_


class ResBlock(nn.Module):
    """
    Convolutional blocks
    """
    def __init__(self, in_channels, out_channels, leaky=True):
        super().__init__()
        # activation, support PReLU and common ReLU
        self.act1 = nn.PReLU() if leaky else nn.ReLU(inplace=True)
        self.act2 = nn.PReLU() if leaky else nn.ReLU(inplace=True)

        self.conv1 = nn.Sequential(
            Normalize(in_channels),
            self.act1,
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True),
        )
        
        self.conv2 = nn.Sequential(
            Normalize(out_channels),
            self.act2,
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True),
        )

        if in_channels == out_channels:
            self.skip_connection = nn.Identity()
        else:
            self.skip_connection = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)


    def forward(self, x):
        h = self.conv1(x)
        h = self.conv2(h)
        return self.skip_connection(x) + h


class ResAttBlock(nn.Module):
    """
    Convolutional blocks
    """
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.resblock  = ResBlock(in_channels=in_channels, out_channels=out_channels)
        self.attention = SpatialSelfAttention(out_channels)


    def forward(self, x):
        h = self.resblock(x)
        h = self.attention(h)
        return h


class ResAttnUNet(nn.Module):
    def __init__(self, in_channel=8, out_channels=8, num_res_blocks=2, ch=32, ch_mult=(1,2,4,4)) -> None:
        super(ResAttnUNet, self).__init__()
        self.ch = ch
        if isinstance(num_res_blocks, int):
            self.num_res_blocks = len(ch_mult) * [num_res_blocks]

        self.input_blocks = nn.Conv2d(in_channel, ch, kernel_size=3, stride=1, padding=1, bias=True)

        self.conv1_0 = ResAttBlock(in_channels=ch,              out_channels=ch * ch_mult[0])
        self.conv2_0 = ResAttBlock(in_channels=ch * ch_mult[0], out_channels=ch * ch_mult[1])
        self.conv3_0 = ResAttBlock(in_channels=ch * ch_mult[1], out_channels=ch * ch_mult[2])
        self.conv4_0 = ResAttBlock(in_channels=ch * ch_mult[2], out_channels=ch * ch_mult[3])

        self.conv3_1 = ResAttBlock(in_channels=ch * (ch_mult[2] + ch_mult[3]), out_channels=ch * ch_mult[2])
        self.conv2_2 = ResAttBlock(in_channels=ch * (ch_mult[1] + ch_mult[2]), out_channels=ch * ch_mult[1])
        self.conv1_3 = ResAttBlock(in_channels=ch * (ch_mult[0] + ch_mult[1]), out_channels=ch * ch_mult[0])
        self.conv0_4 = ResAttBlock(in_channels=ch * (1          + ch_mult[0]), out_channels=ch * 1)

        self.output_blocks = nn.Sequential(
            Normalize(ch*1),
            nn.SiLU(),
            nn.Conv2d(ch*1, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
        )

        self._initialize_weights()
        self._print_networks(verbose=False)

    def forward(self, x) -> torch.Tensor:
        x0 = self.input_blocks(x)
        x1 = self.conv1_0(x0)
        x2 = self.conv2_0(x1)
        x3 = self.conv3_0(x2)
        x4 = self.conv4_0(x3)

        x3_1 = self.conv3_1(torch.cat([x3, x4], dim=1))
        x2_2 = self.conv2_2(torch.cat([x2, x3_1], dim=1))
        x1_3 = self.conv1_3(torch.cat([x1, x2_2], dim=1))
        x0_4 = self.conv0_4(torch.cat([x0, x1_3], dim=1))

        out = dict()
        out['out'] = self.output_blocks(x0_4)
        return out

    def _initialize_weights(self) -> None:
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="leaky_relu")
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def _print_networks(self, verbose=False) -> None:
        logging.info('---------- Networks initialized -------------')
        num_params = 0
        for param in self.parameters():
            num_params += param.numel()
        if verbose:
            logging.info(self.modules())
        logging.info('Total number of parameters : %.3f M' % (num_params / 1e6))
        logging.info('-----------------------------------------------')

In [23]:
class KResAttnUNet_DS(nn.Module):
    def __init__(self, in_channel=8, out_channels=8, num_res_blocks=2, ch=32, ch_mult=(1,2,4,4)) -> None:
        super(KResAttnUNet_DS, self).__init__()
        self.ch = ch
        if isinstance(num_res_blocks, int):
            self.num_res_blocks = len(ch_mult) * [num_res_blocks]

        self.input_blocks = FastKANConvLayer(in_channel, ch, kernel_size=3, stride=1, padding=1, kan_type = "BSpline", num_grids=4, grid_min=-2., grid_max=2.)
        # self.input_blocks = nn.Conv2d(in_channel, ch, kernel_size=3, stride=1, padding=1, bias=True)

        self.conv1_0 = KResAttBlock(in_channels=ch,              out_channels=ch * ch_mult[0])
        self.conv2_0 = KResAttBlock(in_channels=ch * ch_mult[0], out_channels=ch * ch_mult[1])
        self.conv3_0 = KResAttBlock(in_channels=ch * ch_mult[1], out_channels=ch * ch_mult[2])
        self.conv4_0 = KResAttBlock(in_channels=ch * ch_mult[2], out_channels=ch * ch_mult[3])

        self.conv3_1 = KResAttBlock(in_channels=ch * (ch_mult[2] + ch_mult[3]), out_channels=ch * ch_mult[2])
        self.conv2_2 = KResAttBlock(in_channels=ch * (ch_mult[1] + ch_mult[2]), out_channels=ch * ch_mult[1])
        self.conv1_3 = KResAttBlock(in_channels=ch * (ch_mult[0] + ch_mult[1]), out_channels=ch * ch_mult[0])
        self.conv0_4 = KResAttBlock(in_channels=ch * (1          + ch_mult[0]), out_channels=ch * 1)

        self.convds3 = nn.Sequential(Normalize(ch * ch_mult[2]), 
                                    #  nn.SiLU(),
                                     FastKANConvLayer(ch * ch_mult[2], out_channels, kernel_size=3, stride=1, padding=1, kan_type = "BSpline", num_grids=4, grid_min=-2., grid_max=2.)
                                    #  nn.Conv2d(ch * ch_mult[2], out_channels, kernel_size=3, stride=1, padding=1, bias=False)
                                     )
        self.convds2 = nn.Sequential(Normalize(ch * ch_mult[1]), 
                                    #  nn.SiLU(),
                                     FastKANConvLayer(ch * ch_mult[1], out_channels, kernel_size=3, stride=1, padding=1, kan_type = "BSpline", num_grids=4, grid_min=-2., grid_max=2.)
                                    #  nn.Conv2d(ch * ch_mult[1], out_channels, kernel_size=3, stride=1, padding=1, bias=False)
                                    )
        self.convds1 = nn.Sequential(Normalize(ch * ch_mult[0]), 
                                    #  nn.SiLU(),
                                     FastKANConvLayer(ch * ch_mult[0], out_channels, kernel_size=3, stride=1, padding=1, kan_type = "BSpline", num_grids=4, grid_min=-2., grid_max=2.)
                                    #  nn.Conv2d(ch * ch_mult[0], out_channels, kernel_size=3, stride=1, padding=1, bias=False)
                                    )
        self.convds0 = nn.Sequential(Normalize(ch * 1), 
                                    #  nn.SiLU(),
                                     FastKANConvLayer(ch * 1, out_channels, kernel_size=3, stride=1, padding=1, kan_type = "BSpline", num_grids=4, grid_min=-2., grid_max=2.)
                                    #  nn.Conv2d(ch * 1, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
                                     )
        

        self._initialize_weights()
        self._print_networks(verbose=False)

    def forward(self, x) -> torch.Tensor:
        x0 = self.input_blocks(x)
        x1 = self.conv1_0(x0)
        x2 = self.conv2_0(x1)
        x3 = self.conv3_0(x2)
        x4 = self.conv4_0(x3)

        x3_1 = self.conv3_1(torch.cat([x3, x4], dim=1))
        x2_2 = self.conv2_2(torch.cat([x2, x3_1], dim=1))
        x1_3 = self.conv1_3(torch.cat([x1, x2_2], dim=1))
        x0_4 = self.conv0_4(torch.cat([x0, x1_3], dim=1))

        out = dict()
        out['level3'] = self.convds3(x3_1)
        out['level2'] = self.convds2(x2_2)
        out['level1'] = self.convds1(x1_3)
        out['out']    = self.convds0(x0_4)
        return out

    def _initialize_weights(self) -> None:
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="leaky_relu")
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def _print_networks(self, verbose=False) -> None:
        logging.info('---------- Networks initialized -------------')
        num_params = 0
        for param in self.parameters():
            num_params += param.numel()
        if verbose:
            logging.info(self.modules())
        logging.info('Total number of parameters : %.3f M' % (num_params / 1e6))
        logging.info('-----------------------------------------------')

In [24]:
mapping_model = get_cuda(KResAttnUNet_DS(
    in_channel=4, 
    out_channels=4, 
    num_res_blocks=2, 
    ch=32, 
    ch_mult=[1,2,4,4]
))

In [25]:
def dice_coef(pred, target, eps=1e-7):
    # make sure pred has same shape as target
    if pred.shape != target.shape:
        raise ValueError(f"Shape mismatch: pred {pred.shape}, target {target.shape}")

    # if pred are logits -> convert to probabilities
    pred = torch.sigmoid(pred)
    
    # binarize
    pred = (pred > 0.5).float()

    intersection = (pred * target).sum(dim=(1,2,3))
    union = pred.sum(dim=(1,2,3)) + target.sum(dim=(1,2,3))

    dice = (2. * intersection + eps) / (union + eps)
    return dice.mean()

def dice_loss(pred, target):
    return 1 - dice_coef(pred, target)


In [26]:
def compute_iou(pred, target, eps=1e-7):
    pred = (pred > 0.5).float()
    target = (target > 0.5).float()
    intersection = torch.sum(pred * target, dim=list(range(1, pred.ndim)))
    union = torch.sum(pred + target, dim=list(range(1, pred.ndim))) - intersection
    iou = (intersection + eps) / (union + eps)
    return iou.mean().item()

epoch = 300
# Define optimizers
optimizer = torch.optim.AdamW(mapping_model.parameters(), lr=0.002)
scheduler = LinearWarmupCosineAnnealingLR(optimizer, warmup_epochs=5, max_epochs=epoch)

# Define loss functions
mse_loss  = torch.nn.MSELoss(reduction='mean')
dice_loss = DiceLoss()

# For Tensorboard Visualization
iter_num = 0
best_valid_loss = np.inf
best_valid_loss_rec = np.inf
best_valid_dice = 0
best_valid_dice_epoch = 0
best_valid_loss_dice = np.inf
ds_list = ['level2', 'level1', 'out']

# Network training
for epoch in range(1, epoch + 1):
    epoch_start_time = time.time()
    mapping_model.train()

    T_loss = []
    T_loss_Rec = []
    T_loss_Dice = []
    T_IoU = []

    T_loss_valid = []
    T_loss_Rec_valid = []
    T_loss_Dice_valid = []
    T_Dice_valid = []
    T_IoU_valid = []

    ### Training phase
    for batch_data in tqdm(train_dataloader, desc='Train: '):
        img_rgb = batch_data['img']
        img_rgb = 2. * img_rgb - 1.
        seg_raw = batch_data['seg']
        seg_raw = seg_raw.permute(0, 3, 1, 2).float()
        
        # ✅ Binarize (anything > 0 is 1)
        seg_raw = (seg_raw > 0).float()
        
        # For VAE input (scaled to [-1,1])
        seg_rgb = 2. * seg_raw - 1.
        
        # For Dice/IoU (stay in [0,1])
        seg_img = torch.mean(seg_raw, dim=1, keepdim=True)
        # seg_raw = batch_data['seg']
        # # seg_raw = seg_raw.permute(0, 3, 1, 2) / 255.0
        # seg_raw = seg_raw.permute(0, 3, 1, 2)
        # seg_rgb = 2. * seg_raw - 1.
        # seg_img = torch.mean(seg_raw, dim=1, keepdim=True)
        name    = batch_data['name']

        img_latent_mean, img_latent_logvar = get_vae_encoding_mu_and_sigma(vae_model.encode(get_cuda(img_rgb)), scale_factor)
        seg_latent_mean, seg_latent_logvar = get_vae_encoding_mu_and_sigma(vae_model.encode(get_cuda(seg_rgb)), scale_factor)
        img_latent_std = torch.exp(0.5 * img_latent_logvar)
            
        if np.random.uniform() > 0.5:
            img_latent_mean_aug = img_latent_mean + img_latent_std * torch.randn_like(img_latent_std)
        else:
            img_latent_mean_aug = img_latent_mean

        # latent matching
        out_latent_mean_dict = mapping_model(img_latent_mean_aug)
        loss_Rec = get_multi_loss(mse_loss, out_latent_mean_dict, seg_latent_mean, is_ds=True, key_list=ds_list)

        # image matching
        pred_seg_dict = {}
        for level_name in ds_list:
            pred_seg_dict[level_name] = vae_decode(vae_model, out_latent_mean_dict[level_name], scale_factor)

        # if iter_num < 3:  
        #     pred_seg = pred_seg_dict['out']
        # print("Train Debug:")
        # print("seg_raw:", seg_raw.min().item(), seg_raw.max().item())
        # print("seg_img:", seg_img.min().item(), seg_img.max().item())
        # print("pred_seg:", pred_seg.min().item(), pred_seg.max().item())
        loss_Dice = get_multi_loss(dice_loss, pred_seg_dict, get_cuda(seg_img), is_ds=True, key_list=ds_list)

        loss = loss_Rec + loss_Dice

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        iter_num += 1
        if iter_num % 100 == 0:
            print('loss/loss', loss, iter_num)
            print('loss/loss_Rec', loss_Rec, iter_num)
            print('loss/loss_Dice', loss_Dice, iter_num)

        T_loss.append(loss.item())
        T_loss_Rec.append(loss_Rec.item())
        T_loss_Dice.append(loss_Dice.item())

        with torch.no_grad():
            iou_values = []
            for level_name in ds_list:
                pred_seg = pred_seg_dict[level_name]
                iou = compute_iou(pred_seg, get_cuda(seg_img))
                iou_values.append(iou)
            avg_iou = np.mean(iou_values)
            T_IoU.append(avg_iou)

    scheduler.step()
    writer.add_scalar('lr', scheduler.get_last_lr()[0], epoch)

    T_loss = np.mean(T_loss)
    T_loss_Rec = np.mean(T_loss_Rec)
    T_loss_Dice = np.mean(T_loss_Dice)
    T_IoU = np.mean(T_IoU)

    print("Train:")
    print("loss: {:.4f}, loss_Rec: {:.4f}, loss_Dice: {:.4f}, IoU: {:.4f}".format(
        T_loss, T_loss_Rec, T_loss_Dice, T_IoU
    ))
    writer.add_scalar('train/IoU', T_IoU, epoch)

    ### Validation phase
    mapping_model.eval()
    for batch_data in tqdm(valid_dataloader, desc='Valid: '):
        img_rgb = batch_data['img']
        img_rgb = 2. * img_rgb - 1.
        # seg_raw = batch_data['seg']
        # seg_raw = seg_raw.permute(0, 3, 1, 2)
        # # seg_raw = seg_raw.permute(0, 3, 1, 2) / 255.0
        # seg_rgb = 2. * seg_raw - 1.
        # seg_img = torch.mean(seg_raw, dim=1, keepdim=True)

        seg_raw = batch_data['seg']
        seg_raw = seg_raw.permute(0, 3, 1, 2).float()
        
        # ✅ Binarize (anything > 0 is 1)
        seg_raw = (seg_raw > 0).float()
        
        # For VAE input (scaled to [-1,1])
        seg_rgb = 2. * seg_raw - 1.
        
        # For Dice/IoU (stay in [0,1])
        seg_img = torch.mean(seg_raw, dim=1, keepdim=True)
        name    = batch_data['name']

        with torch.no_grad():
            img_latent_mean, img_latent_logvar = get_vae_encoding_mu_and_sigma(vae_model.encode(get_cuda(img_rgb)), scale_factor)
            seg_latent_mean, seg_latent_logvar = get_vae_encoding_mu_and_sigma(vae_model.encode(get_cuda(seg_rgb)), scale_factor)
                
            out_latent_mean_dict = mapping_model(img_latent_mean)
            pred_seg = vae_decode(vae_model, out_latent_mean_dict['out'], scale_factor)
            # if epoch == 1 and len(T_loss_valid) < 2:
            #     print("Valid Debug:")
            #     print("seg_raw:", seg_raw.min().item(), seg_raw.max().item())
            #     print("seg_img:", seg_img.min().item(), seg_img.max().item())
            #     print("pred_seg:", pred_seg.min().item(), pred_seg.max().item())
            loss_Rec = mse_loss(out_latent_mean_dict['out'], seg_latent_mean)
            loss_Dice = dice_loss(pred_seg, get_cuda(seg_img))

            loss = loss_Rec + loss_Dice

            pred_seg = pred_seg.cpu()
            reduce_axis = list(range(1, len(seg_img.shape)))

            intersection = torch.sum(seg_img * pred_seg, dim=reduce_axis)
            y_o = torch.sum(seg_img, dim=reduce_axis)
            y_pred_o = torch.sum(pred_seg, dim=reduce_axis)
            denominator = y_o + y_pred_o
            dice_raw = (2.0 * intersection) / (denominator + 1e-7)
            dice_value = dice_raw.mean()

            iou_value = compute_iou(pred_seg, seg_img)

            T_Dice_valid.append(dice_value.item())
            T_IoU_valid.append(iou_value)
            T_loss_valid.append(loss.item())
            T_loss_Rec_valid.append(loss_Rec.item())
            T_loss_Dice_valid.append(loss_Dice.item())

    T_Dice_valid = np.mean(T_Dice_valid)
    T_IoU_valid = np.mean(T_IoU_valid)
    T_loss_valid = np.mean(T_loss_valid)
    T_loss_Rec_valid = np.mean(T_loss_Rec_valid)
    T_loss_Dice_valid = np.mean(T_loss_Dice_valid)

    print("Valid:")
    print("loss: {:.4f}, loss_Rec: {:.4f}, loss_Dice: {:.4f}, Dice: {:.4f}, IoU: {:.4f}".format(
        T_loss_valid, T_loss_Rec_valid, T_loss_Dice_valid, T_Dice_valid, T_IoU_valid
    ))
    writer.add_scalar('valid/IoU', T_IoU_valid, epoch)

    if T_Dice_valid > best_valid_dice:
        save_name = "best_valid_dice.pth"
        save_checkpoint(mapping_model, save_name, './ckpt/')
        best_valid_dice = T_Dice_valid
        best_valid_dice_epoch = epoch
        print("Save best valid Dice !")

    if T_loss_valid < best_valid_loss:
        save_name = "best_valid_loss.pth"
        save_checkpoint(mapping_model, save_name, './ckpt/')
        best_valid_loss = T_loss_valid
        print("Save best valid Loss All !")

    if T_loss_Rec_valid < best_valid_loss_rec:
        save_name = "best_valid_loss_rec.pth"
        save_checkpoint(mapping_model, save_name, './ckpt/')
        best_valid_loss_rec = T_loss_Rec_valid
        print("Save best valid Loss Rec !")

    if T_loss_Dice_valid < best_valid_loss_dice:
        save_name = "best_valid_loss_dice.pth"
        save_checkpoint(mapping_model, save_name, './ckpt/')
        best_valid_loss_dice = T_loss_Dice_valid
        print("Save best valid Loss Dice !")

    if epoch % 100 == 0:
        save_name = "{}_epoch_{:0>4}.pth".format('latent_mapping_model', epoch)
        save_checkpoint(mapping_model, save_name, './ckpt/')

    print('Current learning rate: {:.5f}'.format(scheduler.get_last_lr()[0]))
    print('epoch %d / %d \t Time Taken: %d sec' % (epoch, epoch, time.time() - epoch_start_time))
    print('best valid dice: {:.4f} at epoch: {}'.format(best_valid_dice, best_valid_dice_epoch))
    print('\n')

Train: 100%|██████████| 42/42 [00:43<00:00,  1.03s/it]


Train:
loss: 58.0691, loss_Rec: 56.8243, loss_Dice: 1.2449, IoU: 0.4430


Valid: 100%|██████████| 40/40 [00:14<00:00,  2.70it/s]


Valid:
loss: 13.7662, loss_Rec: 13.3825, loss_Dice: 0.3838, Dice: 0.6162, IoU: 0.4724
Save best valid Dice !
Save best valid Loss All !
Save best valid Loss Rec !
Save best valid Loss Dice !
Current learning rate: 0.00050
epoch 1 / 1 	 Time Taken: 58 sec
best valid dice: 0.6162 at epoch: 1




Train: 100%|██████████| 42/42 [00:41<00:00,  1.02it/s]


...

Valid:
loss: 0.2749, loss_Rec: 0.2010, loss_Dice: 0.0739, Dice: 0.9261, IoU: 0.8929
Save best valid Loss All !
Save best valid Loss Rec !
Current learning rate: 0.00000
epoch 299 / 299 	 Time Taken: 56 sec
best valid dice: 0.9263 at epoch: 289




Train: 100%|██████████| 42/42 [00:41<00:00,  1.01it/s]


loss/loss tensor(0.7763, device='cuda:0', grad_fn=<AddBackward0>) 12600
loss/loss_Rec tensor(0.5821, device='cuda:0', grad_fn=<AddBackward0>) 12600
loss/loss_Dice tensor(0.1942, device='cuda:0', grad_fn=<AddBackward0>) 12600
Train:
loss: 1.7316, loss_Rec: 1.2840, loss_Dice: 0.4476, IoU: 0.8031


Valid: 100%|██████████| 40/40 [00:14<00:00,  2.70it/s]

Valid:
loss: 0.2749, loss_Rec: 0.2010, loss_Dice: 0.0739, Dice: 0.9261, IoU: 0.8930
Current learning rate: 0.00000
epoch 300 / 300 	 Time Taken: 56 sec
best valid dice: 0.9263 at epoch: 289


